# 实现一个简单的自注意力机制

输入向量 => 上下文向量

1. 计算注意力权重（attention weights）：==通过输入向量==
	1. 通过计算与每个输入的点积来计算：`attn_scores = inputs @ inputs.T`
	2. 然后再进行softmax归一化得到：`attn_weights = torch.softmax(attn_scores, dim=1)`
2. 计算上下文向量（context vector）：==通过注意力权重和输入向量==
	1. `attn_weights @ inputs`

In [1]:
# 生成一个文本 embedding 后的数据：
import torch
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     (x^1)
    [0.55, 0.87, 0.66], # journey  (x^2)
    [0.57, 0.85, 0.64], # starts   (x^3)
    [0.22, 0.58, 0.33], # with     (x^4)
    [0.77, 0.25, 0.10], # one      (x^5)
    [0.05, 0.80, 0.55]] # step     (x^6)   
)

为每个输入元素，计算一个上下文向量，exp：

为输入元素 $x^{(2)}$ 计算上下文向量 $z^{(2)}$ 。

![image.png](https://ressmatthew-picture-cloud-storage.oss-cn-hangzhou.aliyuncs.com/img/202408121511892.png)

中间注意力机制得分：$\alpha_{21}$ - $\alpha_{2T}$：可以通过计算$x^{(2)}$ 与每个输入的点积来计算。

In [6]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [7]:
# 验证 attn_scores_2[0] 即 inputs[0]点积inputs[1]

print(0.43 * 0.55 + 0.15 * 0.87 + 0.89 * 0.66)

0.9544


归一化：获取总和为1的注意力权重

In [8]:
# 简单归一化-总和

attn_weight_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights:", attn_weight_2_tmp)
print("Sum", attn_weight_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum tensor(1.0000)


实际应用一般使用softmax（归一化指数函数）函数进行归一化：

$$\sigma(\mathbf{z})_j=\frac{e^{z_j}}{\sum_{k=1}^Ke^{z_k}}\quad\mathrm{for~}j=1,...,K.$$

![image.png](https://ressmatthew-picture-cloud-storage.oss-cn-hangzhou.aliyuncs.com/img/202408121809147.png)

* softmax函数作用：
    *  在处理极端值时表现更佳
    * 均为正值

In [10]:
def softmax_native(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_native(attn_scores_2)
print("Attention weight:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weight: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [13]:
# 用性能优化了的softmax():

attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [14]:
# 通过输入向量与注意力权重计算上下文向量

query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


上面只是针对 $input_2$ 计算，

下面对所有的输入的注意力权重进行计算：
* 方案1:嵌套for循环
* 方案2:矩阵乘法

In [26]:
# 方案一 - 嵌套for循环：

attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

attn_weights = torch.softmax(attn_scores, dim=1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [27]:
# 方案二 - 矩阵乘法

attn_scores = inputs @ inputs.T
attn_weights = torch.softmax(attn_scores, dim=1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [29]:
print(inputs.shape)
print(attn_weights.shape)

torch.Size([6, 3])
torch.Size([6, 6])


最后将所有输入向量与注意力权重加权求和计算得到上下文向量（context vector）。

In [30]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


## 小结

通过矩阵乘法计算代码如下：

```python
attn_scores = inputs @ inputs.T
# 注意力权重
attn_weights = torch.softmax(attn_scores, dim=1)
all_context_vecs = attn_weights @ inputs
```